In [143]:
'''Здесь взят отфильтрованный по протоколу TCP в Wireshark трафик, здесь записи взаимодействия ботнета и машины.
Аномальные записи были помечены красным и черным в интерфейсе Wireshark, поэтому мне было просто отличить их от чистых.
Сбор датасета Okey был произведен вручную: вырезались из исходного датасета чистые фрагменты и собирались в единый список,
поэтому по таймингу есть пустые места. Используется информация из графы Info.'''
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import re
import pandas as pd

In [145]:
df = pd.read_csv('B:\Учёба\диплом\трафик\Okay.csv')

,Unnamed: 0,No.,Time,Source,Destination,Protocol,Length,Info
0,0,77,132.314700,10.0.0.45,23.62.237.87,TCP,66,49161 > 80 [SYN] Seq=0 Win=8192 Len=0 MSS=14...
1,1,78,132.333025,23.62.237.87,10.0.0.45,TCP,66,"80 > 49161 [SYN, ACK] Seq=0 Ack=1 Win=14600 ..."
2,2,79,132.333444,10.0.0.45,23.62.237.87,TCP,54,49161 > 80 [ACK] Seq=1 Ack=1 Win=66560 Len=0
3,3,80,132.333981,10.0.0.45,23.62.237.87,HTTP,151,GET /ncsi.txt HTTP/1.1
4,4,81,132.352766,23.62.237.87,10.0.0.45,TCP,54,80 > 49161 [ACK] Seq=1 Ack=98 Win=14624 Len=0
...,...,...,...,...,...,...,...,...
52241,52995,54314,10631.567258,23.62.237.95,10.0.0.45,TCP,1506,80 > 49437 [ACK] Seq=1659523 Ack=2968 Win=23...
52242,52996,54315,10631.567593,23.62.237.95,10.0.0.45,TCP,1506,80 > 49437 [ACK] Seq=1660975 Ack=2968 Win=23...
52243,52997,54316,10631.567908,23.62.237.95,10.0.0.45,TCP,1506,80 > 49437 [ACK] Seq=1662427 Ack=2968 Win=23...
52244,52998,54317,10631.568227,23.62.237.95,10.0.0.45,TCP,1506,80 > 49437 [ACK] Seq=1663879 Ack=2968 Win=23...


In [146]:
'''Понадобилось почистить от знаков препинания, чтоб токенизация прошла лучше.'''
def reformat(text):
    return re.sub("[,|<|>|=|\|/|.]",' ',text)
text = [reformat(i) for i in df['Info']]

['49161     80 [SYN] Seq 0 Win 8192 Len 0 MSS 1460 WS 256 SACK_PERM 1',
 '80     49161 [SYN  ACK] Seq 0 Ack 1 Win 14600 Len 0 MSS 1452 SACK_PERM 1 WS 32',
 '49161     80 [ACK] Seq 1 Ack 1 Win 66560 Len 0',
 'GET  ncsi txt HTTP 1 1 ',
 '80     49161 [ACK] Seq 1 Ack 98 Win 14624 Len 0',
 'HTTP 1 1 200 OK  (text plain)',
 '80     49161 [FIN  ACK] Seq 180 Ack 98 Win 14624 Len 0',
 '49161     80 [ACK] Seq 98 Ack 181 Win 66560 Len 0',
 '49161     80 [FIN  ACK] Seq 98 Ack 181 Win 66560 Len 0',
 '80     49161 [ACK] Seq 181 Ack 99 Win 14624 Len 0',
 '49166     443 [SYN] Seq 0 Win 8192 Len 0 MSS 1460 WS 4 SACK_PERM 1',
 '443     49166 [SYN  ACK] Seq 0 Ack 1 Win 14600 Len 0 MSS 1452 SACK_PERM 1 WS 32',
 '49166     443 [ACK] Seq 1 Ack 1 Win 66792 Len 0',
 'Client Hello',
 '443     49166 [ACK] Seq 1 Ack 46 Win 14624 Len 0',
 'Server Hello',
 'Certificate  Server Hello Done',
 '49166     443 [ACK] Seq 46 Ack 2398 Win 66792 Len 0',
 'Client Key Exchange  Change Cipher Spec  Encrypted Handshake Messag

In [147]:
# После очистки проведена токенизация с длиной строки в 20, после чего посмотели на более часто встречающиеся слова.
tokenizer = Tokenizer(num_words = 20)
tokenizer.fit_on_texts(text)
tokenizer.word_index

{'ack': 1,
 'win': 2,
 'seq': 3,
 'len': 4,
 '80': 5,
 '49256': 6,
 'tcp': 7,
 'of': 8,
 'segment': 9,
 'a': 10,
 'reassembled': 11,
 'pdu': 12,
 '1452': 13,
 '42240': 14,
 '0': 15,
 '49437': 16,
 '8077': 17,
 '11017': 18,
 '10723': 19,
 '10429': 20,
 '6313': 21,
 '12487': 22,
 '12193': 23,
 '6019': 24,
 '7783': 25,
 '7489': 26,
 '8371': 27,
 '11311': 28,
 '6607': 29,
 '6901': 30,
 '7195': 31,
 '12781': 32,
 '10135': 33,
 '8665': 34,
 '5725': 35,
 '13075': 36,
 '9547': 37,
 '9841': 38,
 '11605': 39,
 '8959': 40,
 '9253': 41,
 '11899': 42,
 '5431': 43,
 '5137': 44,
 '4552': 45,
 '4844': 46,
 '4260': 47,
 '443': 48,
 '3968': 49,
 '2968': 50,
 '3676': 51,
 '3384': 52,
 '38208': 53,
 '41408': 54,
 '39264': 55,
 '40352': 56,
 '37120': 57,
 '36064': 58,
 '34976': 59,
 '33920': 60,
 '32832': 61,
 '28544': 62,
 '31776': 63,
 '27488': 64,
 '30688': 65,
 '1': 66,
 '23200': 67,
 '13369': 68,
 '29632': 69,
 '3092': 70,
 '49180': 71,
 '66560': 72,
 '26400': 73,
 '274176': 74,
 '281600': 75,
 '27571

In [148]:
# произведена замена слов на индексы и получилась следующая картина - некоторые записи не были распознаны
sequences = tokenizer.texts_to_sequences(text)
new_sequences = sequences[:len(text)]

[[5, 3, 15, 2, 4, 15],
 [5, 1, 3, 15, 1, 2, 4, 15, 13],
 [5, 1, 3, 1, 2, 4, 15],
 [],
 [5, 1, 3, 1, 2, 4, 15],
 [],
 [5, 1, 3, 1, 2, 4, 15],
 [5, 1, 3, 1, 2, 4, 15],
 [5, 1, 3, 1, 2, 4, 15],
 [5, 1, 3, 1, 2, 4, 15],
 [3, 15, 2, 4, 15],
 [1, 3, 15, 1, 2, 4, 15, 13],
 [1, 3, 1, 2, 4, 15],
 [],
 [1, 3, 1, 2, 4, 15],
 [],
 [],
 [1, 3, 1, 2, 4, 15],
 [],
 [],
 [],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [],
 [1, 3, 1, 2, 4, 15],
 [],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [1, 3, 1, 2, 4, 15],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [1, 3, 1, 2, 4, 15],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [],
 [1, 3, 1, 2, 4, 15],
 [3, 15, 2, 4, 15],
 [1, 3, 15, 1, 2, 4, 15, 13],
 [1, 3, 1, 2, 4, 15],
 [],
 [1, 3, 1, 2, 4, 15],
 [],
 [],
 [1, 3, 1, 2, 4, 15],
 [],
 [],
 [],
 [1, 3, 1, 2, 4, 13, 7, 9, 8, 10, 11, 12],
 [],
 [1, 3, 1, 2, 4, 15],
 [],
 [1, 3, 1, 2,

In [149]:
# Здесь не критично, что в массиве new_sequences есть пустые записи, 
# их не будем принимать за аномалии, а просто удалим.
empt = [i for i,x in enumerate(new_sequences) if x ==[] ]

In [150]:
new_text = np.array(text)
new_a = np.delete(text, empt)

array(['49161     80 [SYN] Seq 0 Win 8192 Len 0 MSS 1460 WS 256 SACK_PERM 1',
       '80     49161 [SYN  ACK] Seq 0 Ack 1 Win 14600 Len 0 MSS 1452 SACK_PERM 1 WS 32',
       '49161     80 [ACK] Seq 1 Ack 1 Win 66560 Len 0', ...,
       '80     49437 [ACK] Seq 1662427 Ack 2968 Win 23200 Len 1452 [TCP segment of a reassembled PDU]',
       '80     49437 [ACK] Seq 1663879 Ack 2968 Win 23200 Len 1452 [TCP segment of a reassembled PDU]',
       '80     49437 [ACK] Seq 1665331 Ack 2968 Win 23200 Len 1452 [TCP segment of a reassembled PDU]'],
      dtype='<U173')

In [152]:
sequences = tokenizer.texts_to_sequences(new_a)

In [153]:
'''Дальше формируется обучающий датасет, с маленьким коэффициентом аномальности, от 4 до 8'''
x_train = pad_sequences(sequences, maxlen = 15)

In [154]:
df_normal = pd.DataFrame(new_a, columns = ['tokens'])
df_normal['expect'] = [np.sum(i)/len(i) for i in sequences]

,tokens,expect
0,49161 80 [SYN] Seq 0 Win 8192 Len 0 MSS 14...,7.333333
1,80 49161 [SYN ACK] Seq 0 Ack 1 Win 14600 ...,6.555556
2,49161 80 [ACK] Seq 1 Ack 1 Win 66560 Len 0,4.428571
3,80 49161 [ACK] Seq 1 Ack 98 Win 14624 Len 0,4.428571
4,80 49161 [FIN ACK] Seq 180 Ack 98 Win 146...,4.428571
...,...,...
51905,80 49437 [ACK] Seq 1659523 Ack 2968 Win 23...,7.285714
51906,80 49437 [ACK] Seq 1660975 Ack 2968 Win 23...,7.285714
51907,80 49437 [ACK] Seq 1662427 Ack 2968 Win 23...,7.285714
51908,80 49437 [ACK] Seq 1663879 Ack 2968 Win 23...,7.285714


In [155]:
'''Датасет с аномальными данными собирался по томуже принципу, что и с чистыми, 
здесь повторяются те же действия, что и выше.'''
df_anom = pd.read_csv('B:\Учёба\диплом\трафик\Anomaly.csv')

In [156]:
text_anom = [reformat(i) for i in df_anom['Info']]

In [157]:
tokenizer.fit_on_texts(text_anom)

In [158]:
sequences_anom = tokenizer.texts_to_sequences(text_anom)

In [159]:
empt_anom = [i for i,x in enumerate(sequences_anom) if x ==[] ]

In [160]:
new_text = np.array(text_anom)
new_anom = np.delete(text_anom, empt_anom)

In [161]:
sequences_anom = tokenizer.texts_to_sequences(new_anom)

In [162]:
df_anomaly = pd.DataFrame(new_anom, columns = ['tokens'])
df_anomaly['expect'] = [np.sum(i)/len(i)+15 for i in sequences_anom]

,tokens,expect
0,50635 443 [RST ACK] Seq 487042 Ack 2882 W...,22.125000
1,50635 443 [RST] Seq 487041 Win 0 Len 0,24.166667
2,443 50635 [ACK] Seq 2883 Ack 487042 Win 52...,21.000000
3,[TCP Dup ACK 289185#1] 50673 80 [ACK] Seq ...,19.333333
4,[TCP Dup ACK 289185#2] 50673 80 [ACK] Seq ...,19.333333
...,...,...
2374,60951 80 [FIN ACK] Seq 1 Ack 1 Win 66560 ...,19.428571
2375,60954 80 [FIN ACK] Seq 789 Ack 483 Win 64...,19.428571
2376,60953 80 [FIN ACK] Seq 603 Ack 283 Win 66...,19.428571
2377,[TCP Keep-Alive ACK] 60937 80 [ACK] Seq 59...,19.333333


,tokens,expect
0,49161 80 [SYN] Seq 0 Win 8192 Len 0 MSS 14...,7.333333
1,80 49161 [SYN ACK] Seq 0 Ack 1 Win 14600 ...,6.555556
2,49161 80 [ACK] Seq 1 Ack 1 Win 66560 Len 0,4.428571
3,80 49161 [ACK] Seq 1 Ack 98 Win 14624 Len 0,4.428571
4,80 49161 [FIN ACK] Seq 180 Ack 98 Win 146...,4.428571
...,...,...
51905,80 49437 [ACK] Seq 1659523 Ack 2968 Win 23...,7.285714
51906,80 49437 [ACK] Seq 1660975 Ack 2968 Win 23...,7.285714
51907,80 49437 [ACK] Seq 1662427 Ack 2968 Win 23...,7.285714
51908,80 49437 [ACK] Seq 1663879 Ack 2968 Win 23...,7.285714


,tokens,expect
0,50635 443 [RST ACK] Seq 487042 Ack 2882 W...,22.125000
1,50635 443 [RST] Seq 487041 Win 0 Len 0,24.166667
2,443 50635 [ACK] Seq 2883 Ack 487042 Win 52...,21.000000
3,[TCP Dup ACK 289185#1] 50673 80 [ACK] Seq ...,21.818182
4,[TCP Dup ACK 289185#2] 50673 80 [ACK] Seq ...,20.800000
...,...,...
2411,60951 80 [FIN ACK] Seq 1 Ack 1 Win 66560 ...,23.400000
2412,60954 80 [FIN ACK] Seq 789 Ack 483 Win 64...,19.428571
2413,60953 80 [FIN ACK] Seq 603 Ack 283 Win 66...,19.428571
2414,[TCP Keep-Alive ACK] 60937 80 [ACK] Seq 59...,19.333333


In [164]:
'''После всех преобразований сформирован единый датасет и добавлено время.'''
new_df = pd.concat([df_normal,df_anomaly])

In [165]:
df_timeseries = df['Time'].iloc[:len(new_df)]

In [166]:
new_df['time'] = df_timeseries

In [141]:
new_df.to_csv('B:\Учёба\диплом\трафик\For_lstm.csv')

,tokens,expect,time
0,49161 80 [SYN] Seq 0 Win 8192 Len 0 MSS 14...,7.333333,132.314700
1,80 49161 [SYN ACK] Seq 0 Ack 1 Win 14600 ...,6.555556,132.333025
2,49161 80 [ACK] Seq 1 Ack 1 Win 66560 Len 0,4.428571,132.333444
3,80 49161 [ACK] Seq 1 Ack 98 Win 14624 Len 0,4.428571,132.333981
4,80 49161 [FIN ACK] Seq 180 Ack 98 Win 146...,4.428571,132.352766
...,...,...,...
2374,60951 80 [FIN ACK] Seq 1 Ack 1 Win 66560 ...,19.428571,797.654648
2375,60954 80 [FIN ACK] Seq 789 Ack 483 Win 64...,19.428571,797.655997
2376,60953 80 [FIN ACK] Seq 603 Ack 283 Win 66...,19.428571,797.656781
2377,[TCP Keep-Alive ACK] 60937 80 [ACK] Seq 59...,19.333333,797.657170
